# Importing `pandas`

In [17]:
import pandas as pd

# Reading Data and checking for anomalies

In [18]:
df1 = pd.read_json("anime_full_details.json")
df2 = pd.read_json("anime_list.json")

## Adding ids to both dataframes for easier merging

In [19]:
df1['id'] = df1['link'].apply(lambda x: x.split("/anime/")[1].split("/")[0] if isinstance(x, str) and "/anime/" in x else None)
df2['id'] = df2['link'].apply(lambda x: x.split("/anime/")[1].split("/")[0] if isinstance(x, str) and "/anime/" in x else None)

## Merging the two dataframes

In [20]:
df_merged = pd.merge(left=df1, right=df2, how='right', on=['id'], suffixes=(None, '_y'))
df_merged = df_merged.drop(columns=['title_y', 'link_y', 'score_y']) # drop duplicate columns that were added by the merge

print(f"Full anime data (df1) has size of {df1.shape}.")
print(f"Anime links data (df2) has size of {df2.shape}.")
print(f"Merged table (df_merged) has a size of {df_merged.shape}.")

Full anime data (df1) has size of (28209, 24).
Anime links data (df2) has size of (28210, 5).
Merged table (df_merged) has a size of (28210, 25).



`keep=False` allows to see all instances of duplication for better analysis. 
Apparently during the scraping process, this title below was scraped twice.


In [21]:
df_merged[df_merged['link'].duplicated(keep=False)]

,title,synopsis,type,episodes,status,aired,premiered,broadcast,producers,licensors,...,duration,rating,score,ranked,popularity,members,favorites,link,id,rank
549,Zankyou no Terror,"Painted in red, the word ""VON"" is all that is ...",TV,\n 11\n,\n Finished Airing\n,"\n Jul 11, 2014 to Sep 26, 2014\n",Summer 2014,[\n Fridays at 00:50 (JST)\n ],"[Aniplex, Dentsu, Fuji TV, Tohokushinsha Film ...",[Funimation],...,\n 22 min. per ep.\n,\n R - 17+ (violence & profanity)\n,8.08,\n #551,\n #122\n,"\n 1,220,926\n","\n 22,796\n",https://myanimelist.net/anime/23283/Zankyou_no...,23283,550
550,Zankyou no Terror,"Painted in red, the word ""VON"" is all that is ...",TV,\n 11\n,\n Finished Airing\n,"\n Jul 11, 2014 to Sep 26, 2014\n",Summer 2014,[\n Fridays at 00:50 (JST)\n ],"[Aniplex, Dentsu, Fuji TV, Tohokushinsha Film ...",[Funimation],...,\n 22 min. per ep.\n,\n R - 17+ (violence & profanity)\n,8.08,\n #551,\n #122\n,"\n 1,220,926\n","\n 22,796\n",https://myanimelist.net/anime/23283/Zankyou_no...,23283,551


Let's observe the `df_merged` again, but with dropped duplicates this time.

In [22]:
print(f"Full anime data (df1) has size of {df1.shape}.")
print(f"Anime links data (df2) has size of {df2.shape}.")
print(f"Merged table (df_merged) has a size of {df_merged.shape}.")

df_merged = df_merged.drop_duplicates(subset=['id'])

print(f"Merged table (df_merged) after dropping duplicates has a size of {df_merged.shape}.")

Full anime data (df1) has size of (28209, 24).
Anime links data (df2) has size of (28210, 5).
Merged table (df_merged) has a size of (28210, 25).
Merged table (df_merged) after dropping duplicates has a size of (28209, 25).


As we can see here, duplication was removed, which makes `df_merged` more consistent with anime links data `df2`.

# Cleaning

Let's see the very first title to scale cleaning technique to entire dataframe

In [23]:
df_merged.head(1)

,title,synopsis,type,episodes,status,aired,premiered,broadcast,producers,licensors,...,duration,rating,score,ranked,popularity,members,favorites,link,id,rank
0,Sousou no Frieren,During their decade-long quest to defeat the D...,TV,\n 28\n,\n Finished Airing\n,"\n Sep 29, 2023 to Mar 22, 2024\n",Fall 2023,[\n Fridays at 23:00 (JST)\n ],"[Aniplex, Dentsu, Shogakukan-Shueisha Producti...",[Crunchyroll],...,\n 24 min. per ep.\n,\n PG-13 - Teens 13 or older\n,9.31,\n #1,\n #157\n,"\n 1,060,746\n","\n 65,118\n",https://myanimelist.net/anime/52991/Sousou_no_...,52991,1


In [24]:
df_merged.head(1).T # Transpose for better visibility

,0
title,Sousou no Frieren
synopsis,During their decade-long quest to defeat the D...
type,TV
episodes,\n 28\n
status,\n Finished Airing\n
aired,"\n Sep 29, 2023 to Mar 22, 2024\n"
premiered,Fall 2023
broadcast,[\n Fridays at 23:00 (JST)\n ]
producers,"[Aniplex, Dentsu, Shogakukan-Shueisha Producti..."
licensors,[Crunchyroll]


- I can see that `broadcast` column was scraped as list. This tells me that the original scraper logic had `.getall()` method. Replaced with `.get()` for the next time.

- I can see a lot of `\n`s. Let's use `.apply` method to replace those.

## Clean the `broadcast` column

In [25]:
df_merged['broadcast'] = df_merged['broadcast'].explode()

## Clean the `\n`s from the data.

In [26]:
for i in df_merged.columns:
    df_merged[i] = df_merged[i].apply(lambda x: x.replace("\n", "") if isinstance(x, str) else x)
df_merged.head()

,title,synopsis,type,episodes,status,aired,premiered,broadcast,producers,licensors,...,duration,rating,score,ranked,popularity,members,favorites,link,id,rank
0,Sousou no Frieren,During their decade-long quest to defeat the D...,TV,28,Finished Airing,"Sep 29, 2023 to Mar 22, 2024",Fall 2023,Fridays at 23:00 (JST),"[Aniplex, Dentsu, Shogakukan-Shueisha Producti...",[Crunchyroll],...,24 min. per ep.,PG-13 - Teens 13 or older,9.31,#1,#157,"1,060,746","65,118",https://myanimelist.net/anime/52991/Sousou_no_...,52991,1
1,Fullmetal Alchemist: Brotherhood,After a horrific alchemy experiment goes wrong...,TV,64,Finished Airing,"Apr 5, 2009 to Jul 4, 2010",Spring 2009,Sundays at 17:00 (JST),"[Aniplex, Square Enix, Mainichi Broadcasting S...","[Funimation, Aniplex of America]",...,24 min. per ep.,R - 17+ (violence & profanity),9.10,#2,#3,"3,494,512","232,541",https://myanimelist.net/anime/5114/Fullmetal_A...,5114,2
2,Steins;Gate,Eccentric scientist Rintarou Okabe has a never...,TV,24,Finished Airing,"Apr 6, 2011 to Sep 14, 2011",Spring 2011,Wednesdays at 02:05 (JST),"[Frontier Works, Media Factory, Kadokawa Shote...",[Funimation],...,24 min. per ep.,PG-13 - Teens 13 or older,9.07,#3,#14,"2,676,611","195,034",https://myanimelist.net/anime/9253/Steins_Gate,9253,3
3,Shingeki no Kyojin Season 3 Part 2,Seeking to restore humanity's diminishing hope...,TV,10,Finished Airing,"Apr 29, 2019 to Jul 1, 2019",Spring 2019,Mondays at 00:10 (JST),"[Production I.G, Dentsu, Mainichi Broadcasting...",[Funimation],...,23 min. per ep.,R - 17+ (violence & profanity),9.05,#4,#21,"2,418,640","60,654",https://myanimelist.net/anime/38524/Shingeki_n...,38524,4
4,One Piece Fan Letter,Although the golden age of piracy is about to ...,TV Special,1,Finished Airing,"Oct 20, 2024",None,NaN,[add some],[add some],...,24 min.,PG-13 - Teens 13 or older,9.05,#5,#2281,"96,765","2,034",https://myanimelist.net/anime/60022/One_Piece_...,60022,5


# Changing data types

## Checking data types

In [27]:
df_merged.dtypes

title           object
synopsis        object
type            object
episodes        object
status          object
aired           object
premiered       object
broadcast       object
producers       object
licensors       object
studios         object
source          object
genres          object
demographic     object
themes          object
duration        object
rating          object
score          float64
ranked          object
popularity      object
members         object
favorites       object
link            object
id              object
rank            object
dtype: object

This is how I want it to look:

- title           object 
- synopsis        object 
- type            object 
- `episodes        int64` 
- status          object 
- aired           object 
- premiered       object 
- broadcast       object 
- producers       object 
- licensors       object 
- studios         object 
- source          object 
- genres          object
- demographic     object
- themes          object
- duration        object
- rating          object
- score          float64
- `ranked          int64 `
- `popularity      int64`
- `members         int64`
- `favorites       int64`
- link            object
- `id              int64`
- `rank            int64`

Let's see those columns first:

In [28]:
change_types = ['episodes', 'ranked', 'popularity', 'members', 'favorites', 'id', 'rank']
df_merged[change_types]

,episodes,ranked,popularity,members,favorites,id,rank
0,28,#1,#157,"1,060,746","65,118",52991,1
1,64,#2,#3,"3,494,512","232,541",5114,2
2,24,#3,#14,"2,676,611","195,034",9253,3
3,10,#4,#21,"2,418,640","60,654",38524,4
4,1,#5,#2281,"96,765","2,034",60022,5
...,...,...,...,...,...,...,...
28205,1,N/A,#23413,129,0,58863,-
28206,Unknown,N/A,#18983,353,0,60857,-
28207,Unknown,N/A,#11255,"2,467",10,57969,-
28208,1,N/A,#18697,375,2,53688,-


In [29]:
for i in change_types:
    df_merged[i] = df_merged[i].str.replace("#", "").str.replace(",", "")
    df_merged[i] = pd.to_numeric(df_merged[i], errors="coerce")

df_merged[change_types]

,episodes,ranked,popularity,members,favorites,id,rank
0,28.0,1.0,157,1060746,65118,52991,1.0
1,64.0,2.0,3,3494512,232541,5114,2.0
2,24.0,3.0,14,2676611,195034,9253,3.0
3,10.0,4.0,21,2418640,60654,38524,4.0
4,1.0,5.0,2281,96765,2034,60022,5.0
...,...,...,...,...,...,...,...
28205,1.0,NaN,23413,129,0,58863,NaN
28206,NaN,NaN,18983,353,0,60857,NaN
28207,NaN,NaN,11255,2467,10,57969,NaN
28208,1.0,NaN,18697,375,2,53688,NaN


Decision: Replace all non-number values with `NaN`s

Looks clean. Now we can proceed with further analysis of the data.

# Finding missing values

In [30]:
df_merged.isna().sum()

title              0
synopsis           0
type            4563
episodes         676
status             0
aired              0
premiered      22101
broadcast      19971
producers          0
licensors          0
studios            0
source         20711
genres             0
demographic        0
themes             0
duration           0
rating             0
score           9971
ranked          6661
popularity         0
members            0
favorites          0
link               0
id                 0
rank            6663
dtype: int64

In [31]:
list_columns = [col for col in df_merged.columns if df_merged[col].apply(lambda x: isinstance(x, list)).any()]
print("Columns containing lists:", list_columns)

Columns containing lists: ['producers', 'licensors', 'studios', 'genres', 'demographic', 'themes']


The column names listed abouve will be used to create separate table for SQLite Database.

In [ ]:
def fetch_list_like_columns(df: pd.DataFrame) -> list:
    list_like_columns = []
    for col in df.columns:
        if df[col].apply(lambda x: isinstance(x, list)).any():
            list_like_columns.append(col)
    return list_like_columns
fetch_list_like_columns(df_merged)

['producers', 'licensors', 'studios', 'genres', 'demographic', 'themes']

Things start to get complicated from here so I will describe steps that I went through here:
- TODO:
- 